# Determine whether amoA genes are functional by placing in a tree with known functional genes

#### First run GraftM to find genes

In [ ]:
#!/bin/bash

# load program
module load miniconda3
conda activate graftm_v0.14.0

# set variables
GRAFTM_PKG=/srv/db/graftm/7/7.22.particulate_methane_monoocygenase_pmoA_31082015.gpkg
OUT_DIR="/srv/projects3/sponge_metatranscriptome/graftM"

# run graftM
graftM graft \
--forward ~/Data/MAGs/Das_tool/Das_tool_95/*.fa \
--graftm_package $GRAFTM_PKG \
--threads 20 \
--input_sequence_type nucleotide \
--output_directory $OUT_DIR 

conda deactivate

#### Pull out genes identified by GraftM

In [ ]:
# copy keywords file to graftM directory
cp /srv/projects/marine_sponge/analyses/EMP_DATA/20190702_graftM_amo_searches/EMP_MAG_symlinks/graftm_amo_all_SpongeDb_Mags/TEMP_amo_keywords.txt \
/srv/projects3/sponge_metatranscriptome/graftM

In [10]:
# keywords contains the list of pmoA/amoA to search
cd /srv/projects3/sponge_metatranscriptome/graftM
cat TEMP_amo_keywords.txt

Root; d_Bacteria; Methylococcales_pmoA; Methylocaldum_pmoA
Root; d_Bacteria; Alphaproteobacteria_pmoA; Methylocystaceae_pmoA1
Root; d_Bacteria; Crenothrix_UpmoA
Root; d_Bacteria; pxmA
Root; Archaea_amoA
Root; d_Bacteria; Verrucomicrobia_pmoA
Root; d_Bacteria; Actino_Delta_homologues
Root; d_Bacteria; Alphaproteobacteria_pmoA; Methylocystaceae_pmoA1; f_Methylocystaceae
Root; d_Bacteria; pxmA; Methylocystaceae_pxmA
Root; d_Bacteria; Methylococcales_pmoA


In [ ]:
# find key words (i.e., genes of interest) in GraftM output and save to new file
for DIR in $(ls -d 5*); do
    echo $DIR
    grep -f TEMP_amo_keywords.txt $DIR/${DIR}_read_tax.tsv > $DIR/${DIR}_xmo_read_ids.txt
done

In [ ]:
# clean read ids
# note: The read_ids files give the gene name which can be found in the _orf.fa files. 
# However, it also includes the GraftM hit name, which you have to remove before you can use the xmo_read_ids.txt file to grep the gene names in the orf files
for DIR in $(ls -d 5*); do
    echo $DIR
    cat $DIR/${DIR}_xmo_read_ids.txt | cut -f 1 > $DIR/${DIR}_xmo_read_ids_cut.txt
done

In [ ]:
# Now grep the gene name and sequence from the orf file and concatenate to new file. Use sed to append mag name to end of first line
for DIR in $(ls -d 5*); do
    echo $DIR
    if [[ -f $DIR/${DIR}_orf.fa ]]; then # only execute if orf.fa file is found
        grep -A 1 -f $DIR/${DIR}_xmo_read_ids_cut.txt $DIR/${DIR}_orf.fa | sed "1{s/$/_${DIR}/}" \ # need to use double quote with variables ('' with interpret the input literally)
        >> /srv/projects3/sponge_metatranscriptome/amoA_gene_tree/all_xmo_seqs.fa
    fi
done

In [22]:
# also grep the xmo ids to identify if amo or pmo (without tree)
for DIR in $(ls -d 5*); do
    if [[ -f $DIR/${DIR}_xmo_read_ids.txt ]]; then
        #echo $DIR # to see genome id
        cat $DIR/${DIR}_xmo_read_ids.txt
    fi
done

NODE_2438_length_15059_cov_2.568382_6137_5_123	Root; d_Bacteria; Actino_Delta_homologues
NODE_3107_length_13349_cov_41.222206_5686_1_87	Root; d_Bacteria; Crenothrix_UpmoA
NODE_2_length_1720336_cov_136.089026_778481_5_14320	Root; d_Bacteria; pxmA
NODE_396_length_70699_cov_23.042169_10560_6_195	Root; d_Bacteria; Methylococcales_pmoA
NODE_195_length_141269_cov_26.460691_85276_4_1479	Root; d_Bacteria; Crenothrix_UpmoA
NODE_62_length_335342_cov_11.568420_74186_2_1053	Root; d_Bacteria; Actino_Delta_homologues
NODE_1594_length_37033_cov_105.808805_3487_1_56	Root; d_Bacteria; Actino_Delta_homologues
NODE_2747_length_18020_cov_17.286613_1836_3_45	Root; Archaea_amoA
NODE_44_length_309338_cov_9.455486_156013_4_2920	Root; d_Bacteria; Actino_Delta_homologues
NODE_48_length_280704_cov_28.309643_220369_1_3787	Root; d_Bacteria; Methylococcales_pmoA
NODE_22335_length_1319_cov_2.919304_1190_5_21	Root; d_Bacteria; Methylococcales_pmoA
NODE_2161_length_17070_cov_2.663826_6584_5_140	Root; d_Bacteria; Actin

#### From above:
#### Only amoA is found in Thaumarchetota
512_metabat1_sensitive.077
NODE_2747_length_18020_cov_17.286613_1836_3_45	Root; Archaea_amoA

#### All Desulfobacter are homologues
513_metabat1_sensitive.010
NODE_44_length_309338_cov_9.455486_156013_4_2920	Root; d_Bacteria; Actino_Delta_homologues

501_concoct.007
NODE_2438_length_15059_cov_2.568382_6137_5_123	Root; d_Bacteria; Actino_Delta_homologues

567_concoct.066
NODE_2161_length_17070_cov_2.663826_6584_5_140	Root; d_Bacteria; Actino_Delta_homologues




In [ ]:
# Get previously identified amoA functional and non-functional sequences and homologues for tree
# note: this file also contain amoA genes and homologues from Robbins 2021 paper
# note: using adjusted seqs file as it has formatted names to remove special characters
cp /srv/projects/marine_sponge/analyses/EMP_DATA/20191003_genetree_amoA/all_sequences/all_sequences_adjusted.fasta /srv/projects3/sponge_metatranscriptome/amoA_gene_tree/

# concatenate with our data
cat all_sequences_adjusted.fasta all_xmo_seqs.fa > combined_sequences.fasta

#### Align sequences using mafft

In [ ]:
# load mafft
conda activate mafft-7.455

# align sequences
mafft --maxiterate 1000 --localpair combined_sequences.fasta > combined_sequences_aligned_mafft.fasta

#### Infer tree using IQtree

In [ ]:
# load iqtree
conda activate iqtree_2.1.2  

# run tree inference
iqtree -s combined_sequences_aligned_mafft.fasta -m LG+G -nt AUTO -ntmax 20

#### Export tip labels for ggtree R

In [ ]:
# note: only need tip labls from our MAGs at this point
grep ">" all_xmo_seqs.fa | sed 's/>//g' > amo_tip_labels.txt

# note: amo_label and mag_id identified through read_tax.tsv files and gtdb 